# Introducción

El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

## Preparación de datos

In [30]:
#Cargar todas las librerías
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error 
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.metrics import r2_score
import lightgbm as ltb 

In [2]:
#Cargar los archivos de los datos 
car_data = pd.read_csv('/datasets/car_data.csv')

In [3]:
#Vista general
car_data

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354364,21/03/2016 09:50,0,NaN,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes,21/03/2016 00:00,0,2694,21/03/2016 10:42
354365,14/03/2016 17:48,2200,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN,14/03/2016 00:00,0,39576,06/04/2016 00:46
354366,05/03/2016 19:56,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no,05/03/2016 00:00,0,26135,11/03/2016 18:17
354367,19/03/2016 18:57,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no,19/03/2016 00:00,0,87439,07/04/2016 07:15


In [4]:
#Información general
car_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [5]:
#Vistazo de las estadísticas descriptivas 
car_data.describe(percentiles = np.linspace(0,1,11))

,Price,RegistrationYear,Power,Mileage,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
0%,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
10%,499.000000,1995.000000,0.000000,70000.000000,0.000000,0.0,14959.000000
20%,880.000000,1998.000000,60.000000,100000.000000,2.000000,0.0,26169.000000
30%,1300.000000,2000.000000,75.000000,125000.000000,3.000000,0.0,33611.000000
40%,1900.000000,2001.000000,90.000000,150000.000000,4.000000,0.0,42389.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000


In [6]:
# Número de duplicados 
car_data.duplicated().sum()

262

In [7]:
#Eliminación de valores duplicados
car_data = car_data.drop_duplicates()

In [8]:
# Valores nulos expresados en porcentajes con relación al dataframe completo
car_data.isna().sum()/car_data.shape[0]*100

DateCrawled           0.000000
Price                 0.000000
VehicleType          10.585501
RegistrationYear      0.000000
Gearbox               5.600002
Power                 0.000000
Model                 5.563573
Mileage               0.000000
RegistrationMonth     0.000000
FuelType              9.287871
Brand                 0.000000
NotRepaired          20.091385
DateCreated           0.000000
NumberOfPictures      0.000000
PostalCode            0.000000
LastSeen              0.000000
dtype: float64

### Análisis columnas no categoricas

In [9]:
#Se elimina la columna 'NumberOfPicture'
car_data = car_data.drop('NumberOfPictures',axis=1)

In [10]:
#Se elimanan los valores nulos de estas columnas, pues su porcentaje de valores nulos es bajo
car_data = car_data.dropna(subset=['VehicleType', 'Model', 'Gearbox', 'FuelType'])

In [11]:
#Cantidad de vehículos no reparados versus los reparados
car_data['NotRepaired'].value_counts()

no     217767
yes     27800
Name: NotRepaired, dtype: int64

Los valores ausentes de la columna 'NotRepaired' es significativo, después del análisis se procede a llenar dichas filas con el valor representativo que en este caso es 'no'.

In [12]:
#Llenado de los valores ausentes
car_data['NotRepaired'] = car_data['NotRepaired'].fillna('no')

In [13]:
#Conversión de columnas a tipo fecha y solo tomamos los años
car_data['DateCreated'] = pd.to_datetime(car_data['DateCreated']).dt.to_period('Y')
car_data['DateCrawled'] = pd.to_datetime(car_data['DateCrawled']).dt.to_period('Y')
car_data['LastSeen'] = pd.to_datetime(car_data['LastSeen']).dt.to_period('Y')

### Análisis columnas categóricas

In [14]:
# Análisis columna 'RegistrationYear'
car_data['RegistrationYear'].sort_values().unique()

array([1910, 1919, 1923, 1927, 1928, 1929, 1930, 1931, 1932, 1933, 1934,
       1935, 1936, 1937, 1938, 1942, 1943, 1945, 1947, 1950, 1951, 1952,
       1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963,
       1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974,
       1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985,
       1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996,
       1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007,
       2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018])

Vemos que no hay valores irrelevantes para la creación del modelo

In [15]:
# Análisis de la columna 'Brand'
car_data['Brand'].nunique()

39

Podemos observar que existen 39 marcas, esto haría que el entranamiento del modelo sea demasiado tedioso, por lo tanto solo tomaremos el top 10.

In [16]:
#Número de filas por marca
marcas_pop = car_data.groupby('Brand').count().sort_values(by='Price', ascending=False).head(10)
marcas_pop

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,NotRepaired,DateCreated,PostalCode,LastSeen
Brand,,,,,,,,,,,,,,
volkswagen,60548,60548,60548,60548,60548,60548,60548,60548,60548,60548,60548,60548,60548,60548
bmw,31217,31217,31217,31217,31217,31217,31217,31217,31217,31217,31217,31217,31217,31217
opel,30992,30992,30992,30992,30992,30992,30992,30992,30992,30992,30992,30992,30992,30992
mercedes_benz,27291,27291,27291,27291,27291,27291,27291,27291,27291,27291,27291,27291,27291,27291
audi,24683,24683,24683,24683,24683,24683,24683,24683,24683,24683,24683,24683,24683,24683
ford,20102,20102,20102,20102,20102,20102,20102,20102,20102,20102,20102,20102,20102,20102
renault,13830,13830,13830,13830,13830,13830,13830,13830,13830,13830,13830,13830,13830,13830
peugeot,8906,8906,8906,8906,8906,8906,8906,8906,8906,8906,8906,8906,8906,8906
fiat,7467,7467,7467,7467,7467,7467,7467,7467,7467,7467,7467,7467,7467,7467


In [17]:
marcas = ['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford', 'renault', 'peugeot', 'fiat', 'seat']
car_data['Brand'] = car_data['Brand'].where(car_data['Brand'].isin(marcas), other ='other')
car_data['Brand'].nunique()

11

Ahora podemos ver que que solo existen 10 marcas y las demás se las renombró como otros

In [18]:
#Análisis de la columna Model
car_data['Model'].nunique()

250

Al haber tantos datos únicos en esta columna se procede a eliminarla, ya que tampoco es relevante para el modelo

In [19]:
#Eliminación de la columna Model
car_data = car_data.drop('Model', axis = 1)

In [20]:
#Análisis de la columna Postal Code
car_data['PostalCode'].nunique()

8096

Se encontraron 8096 valores diferentes en esta columna, también se descarta por no ser de mucha importancia

In [21]:
#Eliminación de la columna postal code
car_data = car_data.drop('PostalCode', axis= 1)

Podemos inferir también que las columnas de LastSeen, DateCrawled, DateCreated y Registration Month no tienen incidencia en nuestra variable objetivo que sería el precio

In [22]:
#Eliminación de columnas no relevantes para el modelo
car_data = car_data.drop(['DateCrawled', 'LastSeen', 'RegistrationMonth', 'DateCreated'], axis= 1)

In [23]:
car_data

,Price,VehicleType,RegistrationYear,Gearbox,Power,Mileage,FuelType,Brand,NotRepaired
2,9800,suv,2004,auto,163,125000,gasoline,other,no
3,1500,small,2001,manual,75,150000,petrol,volkswagen,no
4,3600,small,2008,manual,69,90000,gasoline,other,no
5,650,sedan,1995,manual,102,150000,petrol,bmw,yes
6,2200,convertible,2004,manual,109,150000,petrol,peugeot,no
...,...,...,...,...,...,...,...,...,...
354362,3200,sedan,2004,manual,225,150000,petrol,seat,yes
354363,1150,bus,2000,manual,0,150000,petrol,opel,no
354366,1199,convertible,2000,auto,101,125000,petrol,other,no
354367,9200,bus,1996,manual,102,150000,gasoline,volkswagen,no


## Entrenamiento del modelo 

In [24]:
#Separación de los datos en caracteristicas y objetivo
features = car_data.drop('Price', axis=1)
target =car_data['Price']

In [25]:
#Aplicación de OHE para la transformación de características categóricas en numéricas
features_ohe = pd.get_dummies(features,drop_first=True)

#Separamos en conjunto de entrenamiento y validación en un ratio de 75% y 25%
train_features_ohe, valid_features_ohe, train_target_ohe, valid_target_ohe = train_test_split(features_ohe, target,
                                                                              test_size=0.25, random_state=12345 )

## Análisis del modelo

###  Modelo de Regresión Lineal

In [46]:
class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

    
model = LinearRegression()
%time model.fit(train_features_ohe, train_target_ohe)
%time predictions = model.predict(valid_features_ohe)
rmse_lr = round(mean_squared_error(valid_target_ohe, predictions),2)
print('RMSE: ', rmse_lr**0.5)

CPU times: user 115 ms, sys: 65.1 ms, total: 180 ms
Wall time: 105 ms
CPU times: user 10.2 ms, sys: 4.09 ms, total: 14.3 ms
Wall time: 77.8 ms
RMSE:  3041.871935174129


### Modelo de bosque aleatorio de decisión

In [31]:
#Encontramos los mejores parámetros para el modelo
model_rfr =RandomForestRegressor(random_state=12345)
param_grid = {'n_estimators': [1,2,3,4,6,8,10],'max_features': [ 'sqrt', 'log2'],'max_depth' : [2,3,4,5,6,8,10]}
CV_rfr = GridSearchCV(estimator=model_rfr, param_grid=param_grid, cv= 5)
CV_rfr.fit(train_features_ohe,train_target_ohe)
CV_rfr.best_params_

{'max_depth': 10, 'max_features': 'sqrt', 'n_estimators': 10}

In [45]:
#Fijamos el modelo con los parámetros encontrados
model_rfr = RandomForestRegressor(random_state=12345, max_depth=10, max_features= 'sqrt',n_estimators=10)
%time model_rfr.fit(train_features_ohe, train_target_ohe)
%time predicted_valid_rfr = model_rfr.predict(valid_features_ohe)
rmse_rfr = round(mean_squared_error(valid_target_ohe, predicted_valid_rfr),2)
print('RMSE: ', rmse_rfr**0.5)

CPU times: user 1.17 s, sys: 15.9 ms, total: 1.18 s
Wall time: 1.18 s
CPU times: user 80.4 ms, sys: 0 ns, total: 80.4 ms
Wall time: 80.2 ms
RMSE:  2167.103040928142


### Modelo LightGBM

In [44]:
#Probamos los parámetros para el primer modelo
model_lgbm1= ltb.LGBMRegressor(boosting_type='gbdt',num_leaves=15, max_depth=5, learning_rate=0.1, n_estimators=100)
%time model_lgbm1.fit(train_features_ohe, train_target_ohe)
%time predicted_valid_lgbm1 = model_lgbm1.predict(valid_features_ohe)
rmse_lgbm1 = round(mean_squared_error(valid_target_ohe, predicted_valid_lgbm1),2)
print('RMSE: ', rmse_lgbm1**0.5)

CPU times: user 37.6 s, sys: 248 ms, total: 37.8 s
Wall time: 38 s
CPU times: user 511 ms, sys: 0 ns, total: 511 ms
Wall time: 512 ms
RMSE:  1911.2258291473563


In [43]:
#Probamos los parámetros para el segundo modelo
model_lgbm2= ltb.LGBMRegressor(boosting_type='gbdt',num_leaves=25, max_depth=10, learning_rate=0.7, n_estimators=100)
%time  model_lgbm2.fit(train_features_ohe, train_target_ohe)
%time predicted_valid_lgbm2 = model_lgbm2.predict(valid_features_ohe)
rmse_lgbm2 = round(mean_squared_error(valid_target_ohe, predicted_valid_lgbm2),2)
print('RMSE: ', rmse_lgbm2**0.5)

CPU times: user 1min 5s, sys: 520 ms, total: 1min 6s
Wall time: 1min 6s
CPU times: user 439 ms, sys: 0 ns, total: 439 ms
Wall time: 431 ms
RMSE:  1752.11512178852


## Conclusiones

Después de analizar los 3 modelos podemos ver que el LightGBM tiene las mejores predicciones pero a su vez demoran mucho para entrenar y predecir los datos a comparación de los otros.

En este caso podríamos optar por escoger el modelo de Bosque aleatorio ya que sus predicciones no distan tanto del LightGBM pero el tiempo que se toma para el entrenamiento y posterior predicción de los datos es al menos 38 veces menor.